In [1]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from analysis.utils.fileUtilities import loadPath

def recorrerRaiz(raiz, LR = False):
    if LR:
        raiz = raiz[0]

    p = []
    for punto in raiz[0]:
        x = int(punto.attrib['x'])
        y = int(punto.attrib['y'])
        p.append([x,y])
  
    ini = p[0]
    fin = p[-1]

    if ini[1] > fin[1]:
        return p[::-1]
    else:
        return p

def tipAngle(puntos):
    ini = puntos[0]
    fin = puntos[-1]
    hipot = np.sqrt((ini[0]-fin[0])**2 + (ini[1]-fin[1])**2)
    cat = fin[1]-ini[1]

    angle_rad = np.arccos(cat/hipot)
    angle_deg = angle_rad*180/np.pi

    return angle_deg

def emergenceAngle(puntos, largo = 1, pixelsize = 0.04):            
    l = int(largo/pixelsize)
    N = len(puntos)
    ini = puntos[0]
    fin = puntos[min(l, N)-1]

    hipot = np.sqrt((ini[0]-fin[0])**2 + (ini[1]-fin[1])**2)
    cat = fin[1]-ini[1]

    angle_rad = np.arccos(cat/hipot)
    angle_deg = angle_rad*180/np.pi

    return angle_deg


def plotAngles(ax, puntos, angle, tip = True, largo = 2, pixelsize = 0.04):
    if tip:
        ini = puntos[0]
        fin = puntos[-1]
    else:
        l = int(largo/pixelsize)
        N = len(puntos)

        ini = puntos[0]
        fin = puntos[min(l, N)-1]

    xs = ini[0], fin[0]
    ys = ini[1], fin[1]
    ax.plot(xs, ys, linewidth = 1)

    xs = ini[0], fin[0]
    ys = ini[1], fin[1]
    ax.plot([ini[0], ini[0]], ys, linewidth = 1)

    ax.text(x = ini[0] + 10, y = ini[1], s = str(round(angle,2)) + "°", fontsize = 8)

    return

def find_nearest(point, listpoints):
    d = np.linalg.norm(point-listpoints, axis = 1)
    p = np.argmin(d)

    return p, d[p]

def matching(newInis, allNewPoints, oldInis, allOldPoints, oldNames, NRoots = 0):
    # newInis : begin of lateral roots - this run
    # allNewPoints : all points of the lateral root polyline - this run
    
    # oldInis : begin of lateral roots
    # allOldPoints : all points of the lateral root polyline
    # oldNames : names in previous iteration

    matchedNames = []
    matchedInis = []
    matchedPoints = []

    seen = []

    if oldInis == [] or oldNames == []:
        for j in range(0,len(newInis)):
            matchedNames.append('LR%s'%NRoots)
            matchedInis.append(newInis[j])
            matchedPoints.append(allNewPoints[j])
            NRoots += 1
    else:
        op = np.array(oldInis)
        nps = np.array(newInis)

        for j in range(0, nps.shape[0]):
            p, dp = find_nearest(nps[j,:], op)

            if dp < 20:
                matchedNames.append(oldNames[p])
                matchedInis.append(newInis[j])
                matchedPoints.append(allNewPoints[j])
                seen.append(p)
            else:
                matchedNames.append('LR%s'%NRoots)
                matchedInis.append(newInis[j])
                matchedPoints.append(allNewPoints[j])
                NRoots += 1
        
        N_old = op.shape[0]
        seen.sort()

        for j in range(0, N_old):
            if j not in seen:
                matchedNames.append(oldNames[j])
                matchedInis.append(oldInis[j])
                matchedPoints.append(allOldPoints[j])

        idxs = np.argsort(np.array(matchedNames))
        matchedNames = np.array(matchedNames,dtype=object)[idxs].tolist()
        matchedInis = np.array(matchedInis,dtype=object)[idxs].tolist()
        matchedPoints = np.array(matchedPoints,dtype=object)[idxs].tolist()   

    return matchedInis, matchedPoints, matchedNames, NRoots

def lenRoot(points, pixel_size = 0.04):
    accum = 0
    for j in range(1, len(points)):
        d = np.linalg.norm(np.array(points[j]) - np.array(points[j-1]))
        accum += (d * pixel_size)
    
    return accum

def estimateAngles(path, ax, img, i = -1, tip=False):
    plt.ioff()
    paths = loadPath(os.path.join(path,'RSML'))

    LateralRoots = []
    LateralRootsInis = []
    LateralRootsNames = []
    NRoots = 0


    step = paths[i]
    arbol = ET.parse(step).getroot()

    with open(os.path.join(path, 'metadata.json')) as f:
        metadata = json.load(f)

    y1,y2,x1,x2 = metadata['bounding box']
    h = y2-y1
    w = x2-x1

    plant = arbol[1][0][0]
    
    if len(plant) > 1:
        LRs = plant[1:]
        
        pts = []
        inis = []

        ## Primero hago el tracking
        for LR in LRs:
            puntos = recorrerRaiz(LR, True)
            pts.append(puntos)
            inis.append(puntos[0])

        LateralRootsInis, LateralRoots, LateralRootsNames, NRoots = matching(inis, pts, LateralRootsInis, LateralRoots, LateralRootsNames, NRoots)
        
        ## Luego estimo los angulos
        tips = []
        emergs = []
        lengths = []

        for root in LateralRoots:
            TipAngle = tipAngle(root)
            EmergenceAngle = emergenceAngle(root, 2, metadata['pixel_size'])

            tips.append(TipAngle)
            emergs.append(EmergenceAngle)

        ax.imshow(img)
        
        for i in range(0, len(pts)):            
            if not tip:
                plotAngles(ax, pts[i], emergs[i], tip = tip)
            else:
                plotAngles(ax, pts[i], tips[i], tip = tip)
        ax.axis('off')
    
    return ax

In [19]:
import json 
import cv2

plt.ioff()


exp_path = "/media/ngaggion/DATA/Raices/Analisis/NFYA10_ALL/BAMBOO_NFYA10_NFYA2/Analysis"
save_path = "/media/ngaggion/DATA/Raices/Analisis/NFYA10_ALL/BAMBOO_NFYA10_NFYA2/Report/Angles/"
os.makedirs(save_path, exist_ok = True)
experiments = os.listdir(exp_path)

for exp in experiments:
    robot_path = os.path.join(exp_path, exp)

    for robot in os.listdir(robot_path):
        cam_path = os.path.join(robot_path, robot)

        for cam in os.listdir(cam_path):
            plant_path = os.path.join(cam_path, cam)

            for plant in os.listdir(plant_path):
                results_path = os.path.join(plant_path, plant, "Results_0")

                if os.path.exists(results_path):
                    metadata_path = results_path + "/metadata.json"
                    metadata = json.load(open(metadata_path))

                    metadata["ImagePath"] = metadata["ImagePath"].replace("2023/rpi", "2023/Septiembre/rpi")
                    metadata["ImagePath"] = metadata["ImagePath"].replace("Videos2022", "Datasets/Videos2022")

                    image_path = os.listdir(metadata["ImagePath"])
                    
                    i = -1

                    images = sorted([os.path.join(metadata["ImagePath"], image) for image in image_path if image.endswith(".png")])
                    RSML = os.listdir(os.path.join(results_path, "RSML"))
                    images = [image for image in images if image.split('/')[-1].replace('.png','.rsml') in RSML]

                    bbox = metadata["bounding box"]
                    crop = cv2.imread(images[i])[bbox[0]:bbox[1], bbox[2]:bbox[3]]

                    fig, ax = plt.subplots(figsize = (16, 8), dpi = 200)

                    estimateAngles(results_path, ax, crop.copy(), i)
                    plt.title("Emergence Angles")

                    save = exp + "_" + robot + "_" + cam + "_" + plant + "_emergence_angles.svg"
                    plt.savefig(os.path.join(save_path, save), bbox_inches = "tight", dpi = 200)
                    plt.close()
                    plt.clf()
                    plt.cla()

                    print("Saved", save)



Saved WT_rpi22_cam_2_plant_2_emergence_angles.svg
Saved WT_rpi22_cam_2_plant_1_emergence_angles.svg
Saved WT_rpi22_cam_2_plant_4_emergence_angles.svg
Saved WT_rpi22_cam_2_plant_3_emergence_angles.svg
Saved WT_rpi9_cam_2_plant_4_emergence_angles.svg
Saved WT_rpi9_cam_2_plant_3_emergence_angles.svg
Saved WT_rpi24_j_cam_1_plant_2_emergence_angles.svg
Saved WT_rpi24_j_cam_1_plant_4_emergence_angles.svg
Saved WT_rpi24_j_cam_4_plant_2_emergence_angles.svg
Saved WT_rpi24_j_cam_3_plant_2_emergence_angles.svg
Saved WT_rpi24_j_cam_3_plant_4_emergence_angles.svg
Saved WT_rpi9_j_cam_1_plant_3_emergence_angles.svg
Saved WT_rpi9_j_cam_4_plant_1_emergence_angles.svg
Saved WT_rpi9_j_cam_4_plant_3_emergence_angles.svg
Saved WT_rpi9_j_cam_2_plant_1_emergence_angles.svg
Saved WT_rpi9_j_cam_3_plant_1_emergence_angles.svg
Saved WT_rpi9_j_cam_3_plant_3_emergence_angles.svg
Saved WT_rpi17_j_cam_1_plant_2_emergence_angles.svg
Saved WT_rpi17_j_cam_4_plant_2_emergence_angles.svg
Saved WT_rpi17_j_cam_3_plant_4_e